In [34]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point
import numpy as np

from iduedu import get_intermodal_graph, get_adj_matrix_gdf_to_gdf
import networkx as nx

In [6]:
context_ter = gpd.read_file('context_ter.geojson')
natural = gpd.read_file('shl_natural_estimated.geojson')
accomodation = gpd.read_file('accommodation.geojson')
attractions = gpd.read_file('attractions.geojson')

In [39]:
natural = natural.explode()

C:\Users\thebe\AppData\Local\Temp\ipykernel_27140\2226234898.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  natural = natural.explode()


In [40]:
natural.reset_index(drop=True, inplace=True)

In [41]:
attractions_gdf = pd.concat([natural, attractions])

In [7]:
G_intermodal = get_intermodal_graph(polygon=context_ter.unary_union, clip_by_bounds=True)

2024-10-20 20:46:06.116 | INFO     | iduedu.modules.drive_walk_builder:get_walk_graph:214 - Downloading walk graph from OSM, it may take a while for large territory ...
Parsing pt routes:   0%|          | 0/60 [00:00<?, ?it/s]c:\Projects\IDU\PriOC\.venv\Lib\site-packages\iduedu\modules\routes_parser.py:264: UserWarning: В одном из маршрутов нет пути, пока не работаю с этим
  warnings.warn("В одном из маршрутов нет пути, пока не работаю с этим", UserWarning)
c:\Projects\IDU\PriOC\.venv\Lib\site-packages\shapely\linear.py:88: RuntimeWarning: invalid value encountered in line_locate_point
  return lib.line_locate_point(line, other)
Parsing pt routes: 100%|██████████| 60/60 [00:04<00:00, 12.60it/s]
2024-10-20 20:46:15.993 | INFO     | iduedu.modules.pt_walk_joiner:join_pt_walk_graph:50 - Composing intermodal graph...


In [33]:
attractions_gdf.to_crs(G_intermodal.graph['crs'], inplace=True)

In [10]:
nodes_list = []
for data in list(G_intermodal.nodes.data()):
    current_dict = {
        'id': data[0]
    }
    geometry = Point([data[1]['x'], data[1]['y']])
    current_dict['geometry'] = geometry
    current_dict.update(data[1])
    current_dict.pop('x')
    current_dict.pop('y')
    nodes_list.append(current_dict)

In [11]:
nodes_df = pd.DataFrame(nodes_list)

In [12]:
nodes_gdf = gpd.GeoDataFrame(nodes_df, geometry='geometry', crs=32636)

In [14]:
edges_list = []
for data in list(G_intermodal.edges.data()):
    current_dict = {
        'id': (data[0], data[1])
    }
    current_dict.update(data[2])
    edges_list.append(current_dict)
    
graph_gdf = gpd.GeoDataFrame(
    data=edges_list, geometry='geometry', crs=32636
)

In [16]:
full_gdf = pd.concat([nodes_gdf, graph_gdf])

In [19]:
G_intermodal.graph

{'crs': 32636, 'type': 'intermodal', 'walk_speed': 83.33333333333333}

In [47]:
accomodation.to_crs(G_intermodal.graph['crs'], inplace=True)
attractions_gdf.to_crs(G_intermodal.graph['crs'], inplace=True)
accomodation['index'] = accomodation.index.copy()
attractions_gdf['index'] = attractions_gdf.index.copy()

In [66]:
adjency_matrix = get_adj_matrix_gdf_to_gdf(attractions_gdf, accomodation, G_intermodal, weight='time_min', dtype=np.float16)
for column in list(adjency_matrix.columns):
    adjency_matrix[column] = adjency_matrix[column].apply(lambda x: round(x, 2))

In [76]:
valuable = attractions_gdf[attractions_gdf['value'] > 5]['index'].to_list()

In [82]:
valuable_matrix = adjency_matrix[adjency_matrix.index.isin(valuable)]

In [85]:
for hotel in list(valuable_matrix.columns):
    if valuable_matrix[hotel].sum() + 20 * (len(valuable_matrix)) > 14 * 60:
        print("Can't fit in one day")

Can't fit in one day
Can't fit in one day
Can't fit in one day
Can't fit in one day
Can't fit in one day
Can't fit in one day
Can't fit in one day
Can't fit in one day
Can't fit in one day
Can't fit in one day
Can't fit in one day
Can't fit in one day


In [94]:
from random import randint
accomodation['capacity'] = [randint(10, 40) for i in range(len(accomodation))]

In [95]:
accomodation['capacity']

0     21
1     24
2     24
3     18
4     18
5     31
6     39
7     36
8     17
9     12
10    37
11    19
Name: capacity, dtype: int64

In [106]:
attractions_gdf['generated_demand'] = attractions_gdf['value'].apply(lambda x: x * 2 if x > 5 else x)

In [107]:
attractions_gdf

,natural,geom_type,value,geometry,name,tourism,index,generated_demand
0,grassland,None,1,"POLYGON ((384357.584 6632455.180, 384375.780 6...",NaN,NaN,0,1
1,grassland,None,1,"POLYGON ((384714.343 6632536.854, 384711.651 6...",NaN,NaN,1,1
2,grassland,None,1,"POLYGON ((383992.860 6632665.664, 383996.078 6...",NaN,NaN,2,1
3,grassland,None,1,"POLYGON ((384536.721 6633013.623, 384567.944 6...",NaN,NaN,3,1
4,grassland,None,1,"POLYGON ((385204.245 6633861.213, 385121.219 6...",NaN,NaN,4,1
...,...,...,...,...,...,...,...,...
74,NaN,NaN,4,"POLYGON ((390334.585 6647892.701, 390425.522 6...",None,attraction,74,4
75,NaN,NaN,5,"POLYGON ((390469.696 6647817.343, 390468.489 6...",None,attraction,75,5
76,NaN,NaN,8,"POLYGON ((390425.522 6647965.888, 390448.595 6...",None,attraction,76,16
77,NaN,NaN,8,"POLYGON ((389969.691 6646934.272, 389965.729 6...",Петровский шлюз,attraction,77,16


In [110]:
valuable_demand = attractions_gdf[attractions_gdf['value'] > 5]['generated_demand'].sum()

In [111]:
valuable_demand

618

In [103]:
valuable_demand

1545